In [7]:
import csv
import os
import shap
import numpy as np
import pandas as pd
import pickle as pkl
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold, cross_val_predict, train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, auc, precision_recall_fscore_support
from sklearn.decomposition import PCA
from tqdm import tqdm

In [8]:
#UTF-8 encoding issue

def pkl_dump(data, file):
    with open(file, "wb") as fw:
        pkl.dump(data, fw)

        
def pkl_load(file):
    with open(file, "rb") as fr:
        data = pkl.load(fr)
    return data

def pkl4_dump(data, file):
    with open(file, "wb") as fw:
        pkl.dump(data, fw, pkl.HIGHEST_PROTOCOL)

        
def pkl4_load(file):
    with open(file, "rb") as fr:
        data = pkl.load(fr)
    return data

In [9]:
os.chdir('/mnt/data1/songziwu/data/crc_data/')
os.getcwd()

'/mnt/data1/songziwu/data/crc_data'

Add performance report on training data cross-validation

In [10]:
def expr(clf, params, tasks, nb, nit, model_name='LR'):
    for task in tasks:
        print(f"current task: {task} {model_name}")
        res_output = f"{task}year_{model_name}.txt"
        model_dump = f"{task}year_{model_name}_model.pkl"

        fea2iD, features = pkl_load(f"./03_MQ_Encoding_Files/data_RC{task}yr_expr_features.pkl")
        tr_dx, tr_dy, ts_dx, ts_dy = pkl4_load(f"./03_MQ_Encoding_Files/expr_data_RC{task}yr_train_test.pkl")
        print(tr_dx.shape, ts_dx.shape)
        
        cv_model = RandomizedSearchCV(clf, params, scoring='roc_auc', n_jobs=nb, 
                                      cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=13), 
                                      verbose=1, iid=True, n_iter=nit, random_state=13)
        cv_model.fit(tr_dx, tr_dy)
        best_5_cv = cv_model.best_estimator_
        opt_clf = cv_model.best_estimator_
        pkl_dump(opt_clf, model_dump)

        preds = opt_clf.predict_proba(ts_dx)
        pkl_dump(preds, f"{task}year_{model_name}_preds.pkl")

        idx = np.argmax(opt_clf.classes_)
        preds_1 = list(map(lambda x: x[idx], preds))

        auc_score = roc_auc_score(ts_dy, preds_1)
        fprs, tprs, ths = roc_curve(ts_dy, preds_1)
        print("auc_score is : ",auc_score)
        
        J_idx = np.argmax(tprs - fprs)
        fpr, tpr, th = fprs[J_idx], tprs[J_idx], ths[J_idx]
        auc_score1 = auc(fprs, tprs)

        sen = tpr
        spe = 1 - fpr
        stats = [sen, spe, auc_score1]
        
#         shap.initjs()
#         explainer = shap.TreeExplainer(opt_clf)
#         shap_values = explainer.shap_values(tr_dx)
#         top10 = shap_values[:10,:]
        
#         with open(f"master_{task}gbt_RCstats.csv", "a") as fp:
#             wr = csv.writer(fp, dialect='excel')
#             wr.writerow(['sen','spef','auc'])
#             wr.writerow(stats)
        
       
#         with open(f"master_{task}gbt_RCshap.csv","a") as sp:
#             sp = csv.writer(ir, dialect='excel')
#             sp.writerow(['feature','shap_value'])
#             sp.writerows([top10])
            

#         with open(res_output, "w") as f:
#             f.write(f'''
# auc1: {auc_score}
# auc2: {auc_score1}
# sensitivity: {sen}
# specificity: {spe}
# J: {th}
#             ''')

        # feature importance (top 1000)
#             imp_feas = opt_clf.feature_importances_
#             imp_feas = list(imp_feas)
#             imp_feas = list(zip(imp_feas, range(len(imp_feas))))
#             imp_feas_500 = sorted(imp_feas, key=lambda x: abs(x[0]), reverse=True)[:1000]
#              with open(f"{task}year_{model_name}_top10features.csv", "w") as f:
#                  for each in imp_feas_500:
#                     imp = each[0]
#                     sfea = features[each[1]]
#                     f.write(f"{sfea},{imp}\n")

In [11]:
def to_matrix(all_data, col_num, num_fea_cols):
    pids = []
    matrix = []
    for idx, d in enumerate(all_data):
        m = np.zeros(col_num + 1)
        mp = []
        if num_fea_cols == -1:
            d1 = d
        else:
            d1 = d[:-num_fea_cols]
        for i, e in enumerate(d1):
            if i == 0:
                m[0] = e
            else:
                m[e] = 1.
        
        if num_fea_cols == -1:
            d2 = []
        else:
            d2 = d[-num_fea_cols:]
        for e in d2:
            mp.append(e)
        
        pids.append(idx)
        nmn = np.concatenate((m, np.array(mp)))
        matrix.append(nmn)
    print(np.array(matrix).shape)
    print(matrix[1])
    return matrix, pids

def imputation(matrix, pids, findings=True, labs=True):
    cols = ['label'] + features
    cols_to_imp = []
    if findings:
        cols += findings_list
        cols_to_imp += findings_list
    
    if labs:
        cols += valid_loinc
        cols_to_imp += valid_loinc
        
    df = pd.DataFrame(data=matrix, index=pids, columns=cols)
#     print(df.head())
#     cau = dict()
#     cou = dict()
#     for col in cols_to_imp:
#         cau[col] = [e for e in set(df[df['label']==1.0][col]) if not pd.isna(e)]
#         cou[col] = [e for e in set(df[df['label']==0.0][col]) if not pd.isna(e)]
        
    np.random.seed(47)
    for col in cols_to_imp:
        s = list(set([e for e in df[col] if not np.isnan(e)]))
        for idx, c in enumerate(df[col]):
            if pd.isna(c):
                # choose case or control 
#                 if df['label'][idx] == 1.0:
#                     s = cau[col]
#                 elif df['label'][idx] == 0.0:
#                     s = cou[col]


                df[col][idx] = np.random.choice(s, 1)
                
#     print(df.head())
    matrix = np.array(df)
    print(matrix.shape, matrix[0])
    return matrix, cols[1:]

def create_data(matrix):
    np.random.seed(13)
    np.random.shuffle(matrix)
    np.random.seed(47)
    np.random.shuffle(matrix)
    dx = []
    dy = []
    for each in matrix:
        dx.append(each[1:])
        dy.append(each[0])
    dx = np.array(dx)
    dy = np.array(dy)
    print(dx.shape, dy.shape)
    return dx, dy

In [12]:
tuned_parameters = {
    'num_leaves': [2,4,8,16,32,64,80,96,128],
    'min_data_in_leaf': [2,8,16,32,64,128,256,512,1024],
    'max_depth': [-1, 8, 16, 32, 64],
    'learning_rate': [0.1, 0.01, 0.001, 0.0001, 0.00001],
    'num_iterations': range(100, 2000, 100),
    'n_estimators': range(100, 2000, 100),
    'subsample_for_bin': range(20000, 300000, 20000),
    'min_split_gain': [0., 1e-3, 1e-4, 1e-5],
    'min_child_weight': [1e-3, 1e-4, 1e-5],
    'min_child_samples': [2,4,8,16,32,64,128],
    'subsample': np.arange(0.5, 1, 0.1),
    'colsample_bytree': np.arange(0.5, 1, 0.1),
    'reg_alpha': np.arange(0, 1, 0.1),
    'reg_lambda': np.arange(0, 1, 0.1),
    'tree_learner': ['feature', 'data', 'voting'],
    'max_bin': [128, 256, 512],
    'boosting_type': ['dart', 'gbdt'],
    'feature_fraction': np.arange(0.5, 1, 0.1), 
    'bagging_fraction': np.arange(0.5, 1, 0.1)
    }

In [13]:
clf = lgb.LGBMClassifier(objective='binary', 
                         n_jobs=10, 
                         scale_pos_Weight=4, use_missing=True)

# 0 YEAR

In [14]:
ptIDs = pd.read_csv("01_MQ_Incident_Match_Files/matched_case_control_RC_01yr.csv",usecols=['PATID'],dtype =str)
ptIDs.head()
fea2id, features = pkl_load("./03_MQ_Encoding_Files/data_RC0yr_expr_features.pkl")
data = pkl_load("./03_MQ_Encoding_Files/data_RC0yr_expr.pkl")

In [15]:
for i in tqdm(range(5)):
    train_id, test_id = train_test_split(ptIDs,test_size=0.2)
    test_id.head(), train_id.head(), test_id.shape, train_id.shape
    test_ids = test_id.PATID.to_list()
    train_ids = train_id.PATID.to_list()

    #print(data[0][0])

    trains = []
    tests = []
    count = 0
    for dp in data:
        pid = dp[0]
        ndata = dp[1:]
        if pid in train_ids:
            trains.append(ndata)
        elif pid in test_ids:
            tests.append(ndata)
        else:
            count = count + 1

    #print(count)
    matrix, pids = to_matrix(trains, len(fea2id), -1)

    tr_dx, tr_dy = create_data(matrix)

    matrix, pids = to_matrix(tests, len(fea2id), -1)

    ts_dx, ts_dy = create_data(matrix)


    pkl4_dump((tr_dx, tr_dy, ts_dx, ts_dy), "./03_MQ_Encoding_Files/expr_data_RC0yr_train_test.pkl")


##Algorithm
    expr(clf, tuned_parameters, tasks = [0], nb = 1, nit = 20, model_name = 'GBT')

  0%|          | 0/5 [00:00<?, ?it/s]

(2129, 7832)
[0. 0. 1. ... 0. 0. 0.]
(2129, 7831) (2129,)
(533, 7832)
[0. 0. 1. ... 0. 0. 0.]
(533, 7831) (533,)
current task: 0 GBT
(2129, 7831) (533, 7831)
Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Found `num_iterations` in params. Will use it instead of argument


[LightGBM] [Warning] Unknown parameter: scale_pos_Weight
[LightGBM] [Warning] bagging_fraction is set=0.8999999999999999, subsample=0.7999999999999999 will be ignored. Current value: bagging_fraction=0.8999999999999999
[LightGBM] [Warning] min_data_in_leaf is set=2, min_child_samples=8 will be ignored. Current value: min_data_in_leaf=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=0.7999999999999999 will be ignored. Current value: feature_fraction=0.7


Found `num_iterations` in params. Will use it instead of argument


...

[LightGBM] [Warning] Unknown parameter: scale_pos_Weight
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=0.8999999999999999 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=64, min_child_samples=8 will be ignored. Current value: min_data_in_leaf=64
[LightGBM] [Warning] feature_fraction is set=0.8999999999999999, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.8999999999999999


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 119.0min finished
Found `num_iterations` in params. Will use it instead of argument


[LightGBM] [Warning] Unknown parameter: scale_pos_Weight
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=0.7999999999999999 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=8, min_child_samples=64 will be ignored. Current value: min_data_in_leaf=8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.7999999999999999 will be ignored. Current value: feature_fraction=0.5


100%|██████████| 5/5 [10:49:00<00:00, 7953.22s/it] 

auc_score is :  0.7477084730605859


# 1 YEAR

In [ ]:
fea2id, features = pkl_load("./03_MQ_Encoding_Files/data_RC1yr_expr_features.pkl")
data = pkl_load("./03_MQ_Encoding_Files/data_RC1yr_expr.pkl")

In [ ]:
for i in range(100):
    train_id, test_id = train_test_split(ptIDs,test_size=0.2)
    test_id.head(), train_id.head(), test_id.shape, train_id.shape
    test_ids = test_id.PATID.to_list()
    train_ids = train_id.PATID.to_list()

# split train and test
    print(data[0][0])
#print(len(data))
    trains = []
    tests = []
    count = 0
    for dp in data:
        pid = dp[0]
        ndata = dp[1:]
        if pid in train_ids:
            trains.append(ndata)
        elif pid in test_ids:
            tests.append(ndata)
        else:
        #print(f"{pid} is not an id in train test")
        count = count + 1
# coln = len(valid_loinc) + len(findings_list)
    print(count)
    matrix, pids = to_matrix(trains, len(fea2id), -1)
# matrix, feas = imputation(matrix, pids)
    tr_dx, tr_dy = create_data(matrix)

    matrix, pids = to_matrix(tests, len(fea2id), -1)
# matrix, feas = imputation(matrix, pids)
    ts_dx, ts_dy = create_data(matrix)

# pkl_dump(feas, "./expr_data_5yr_features.pkl")
    pkl4_dump((tr_dx, tr_dy, ts_dx, ts_dy), "./03_MQ_Encoding_Files/expr_data_RC1yr_train_test.pkl")
# pkl4_dump((), "./expr_data_5yr_test.pkl")
    
    expr(clf, tuned_parameters, tasks = [3,5], nb = 1, nit = 50, model_name = 'GBT')

# 3 YEAR

In [ ]:
ptIDs = pd.read_csv("01_MQ_Incident_Match_Files/matched_case_control_RC_3yr.csv",usecols=['PATID'],dtype =str)
ptIDs.head()
fea2id, features = pkl_load("./03_MQ_Encoding_Files/data_RC3yr_expr_features.pkl")
data = pkl_load("./03_MQ_Encoding_Files/data_RC3yr_expr.pkl")

In [ ]:
for i in range(100):
    train_id, test_id = train_test_split(ptIDs,test_size=0.2)
    test_id.head(), train_id.head(), test_id.shape, train_id.shape
    test_ids = test_id.PATID.to_list()
    train_ids = train_id.PATID.to_list()

# split train and test
    print(data[0][0])
#print(len(data))
    trains = []
    tests = []
    count = 0
    for dp in data:
        pid = dp[0]
        ndata = dp[1:]
        if pid in train_ids:
            trains.append(ndata)
        elif pid in test_ids:
            tests.append(ndata)
        else:
        #print(f"{pid} is not an id in train test")
            count = count + 1
# coln = len(valid_loinc) + len(findings_list)
    print(count)
    matrix, pids = to_matrix(trains, len(fea2id), -1)
# matrix, feas = imputation(matrix, pids)
    tr_dx, tr_dy = create_data(matrix)

    matrix, pids = to_matrix(tests, len(fea2id), -1)
# matrix, feas = imputation(matrix, pids)
    ts_dx, ts_dy = create_data(matrix)

# pkl_dump(feas, "./expr_data_5yr_features.pkl")
    pkl4_dump((tr_dx, tr_dy, ts_dx, ts_dy), "./03_MQ_Encoding_Files/expr_data_RC3yr_train_test.pkl")
# pkl4_dump((), "./expr_data_5yr_test.pkl")
    expr(clf, tuned_parameters, tasks = [3,5], nb = 1, nit = 50, model_name = 'GBT')

# 5 YEAR

In [ ]:
ptIDs = pd.read_csv("01_MQ_Incident_Match_Files/matched_case_control_RC_5yr.csv",usecols=['PATID'],dtype =str)
ptIDs.head()
fea2id, features = pkl_load("./03_MQ_Encoding_Files/data_RC5yr_expr_features.pkl")
data = pkl_load("./03_MQ_Encoding_Files/data_RC5yr_expr.pkl")

In [ ]:
for i in range(100):
    train_id, test_id = train_test_split(ptIDs,test_size=0.2)
    test_id.head(), train_id.head(), test_id.shape, train_id.shape
    test_ids = test_id.PATID.to_list()
    train_ids = train_id.PATID.to_list()

# split train and test
    print(data[0][0])
#print(len(data))
    trains = []
    tests = []
    count = 0
    for dp in data:
        pid = dp[0]
        ndata = dp[1:]
        if pid in train_ids:
            trains.append(ndata)
        elif pid in test_ids:
            tests.append(ndata)
        else:
        #print(f"{pid} is not an id in train test")
            count = count + 1
# coln = len(valid_loinc) + len(findings_list)
    print(count)
    matrix, pids = to_matrix(trains, len(fea2id), -1)
# matrix, feas = imputation(matrix, pids)
    tr_dx, tr_dy = create_data(matrix)

    matrix, pids = to_matrix(tests, len(fea2id), -1)
# matrix, feas = imputation(matrix, pids)
    ts_dx, ts_dy = create_data(matrix)

# pkl_dump(feas, "./expr_data_5yr_features.pkl")
    pkl4_dump((tr_dx, tr_dy, ts_dx, ts_dy), "./03_MQ_Encoding_Files/expr_data_RC5yr_train_test.pkl")
# pkl4_dump((), "./expr_data_5yr_test.pkl")
    expr(clf, tuned_parameters, tasks = [3,5], nb = 1, nit = 50, model_name = 'GBT')

## Untuned model

In [17]:


clf = lgb.LGBMClassifier(objective='binary', 
                         n_jobs=10, 
                         scale_pos_Weight=4, use_missing=True)

def expr(clf, tasks, model_name):

    for task in tasks:
        print(f"current task: {task} {model_name}")
        res_output = f"{task}year_{model_name}.txt"
        model_dump = f"{task}year_{model_name}_model.pkl"

        fea2iD, features = pkl_load(f"./03_MQ_Encoding_Files/data_RC{task}yr_expr_features.pkl")
        tr_dx, tr_dy, ts_dx, ts_dy = pkl4_load(f"./03_MQ_Encoding_Files/expr_data_RC{task}yr_train_test.pkl")

        

        clf.fit(tr_dx, tr_dy)

        pkl_dump(clf, model_dump)

        preds = clf.predict_proba(ts_dx)
        pkl_dump(preds, f"{task}year_{model_name}_preds.pkl")

        idx = np.argmax(clf.classes_)
        preds_1 = list(map(lambda x: x[idx], preds))

        auc_score = roc_auc_score(ts_dy, preds_1)
        fprs, tprs, ths = roc_curve(ts_dy, preds_1)
        print("auc_score is : ",auc_score)

#         J_idx = np.argmax(tprs - fprs)
#         fpr, tpr, th = fprs[J_idx], tprs[J_idx], ths[J_idx]
#         auc_score1 = auc(fprs, tprs)

        return auc_score






for i in tqdm(range(5)):
    train_id, test_id = train_test_split(ptIDs,test_size=0.2)
    test_id.head(), train_id.head(), test_id.shape, train_id.shape
    test_ids = test_id.PATID.to_list()
    train_ids = train_id.PATID.to_list()

    #print(data[0][0])

    trains = []
    tests = []
    count = 0
    for dp in data:
        pid = dp[0]
        ndata = dp[1:]
        if pid in train_ids:
            trains.append(ndata)
        elif pid in test_ids:
            tests.append(ndata)
        else:
            count = count + 1

    #print(count)
    matrix, pids = to_matrix(trains, len(fea2id), -1)

    tr_dx, tr_dy = create_data(matrix)

    matrix, pids = to_matrix(tests, len(fea2id), -1)

    ts_dx, ts_dy = create_data(matrix)


    pkl4_dump((tr_dx, tr_dy, ts_dx, ts_dy), "./03_MQ_Encoding_Files/expr_data_RC0yr_train_test.pkl")


##Algorithm
    expr(clf, tasks = [0], model_name = 'GBT')
    




  0%|          | 0/5 [00:00<?, ?it/s]

(2129, 7832)
[0. 0. 1. ... 0. 0. 0.]
(2129, 7831) (2129,)
(533, 7832)
[0. 0. 1. ... 0. 0. 0.]
(533, 7831) (533,)
current task: 0 GBT
[LightGBM] [Warning] Unknown parameter: scale_pos_Weight



 20%|██        | 1/5 [00:07<00:29,  7.29s/it]

auc_score is :  0.7456405097250167
(2129, 7832)
[0. 0. 1. ... 0. 0. 0.]
(2129, 7831) (2129,)
(533, 7832)
[0. 0. 1. ... 0. 0. 0.]
(533, 7831) (533,)
current task: 0 GBT
[LightGBM] [Warning] Unknown parameter: scale_pos_Weight



 40%|████      | 2/5 [00:12<00:19,  6.59s/it]

...

auc_score is :  0.7456405097250167
(2129, 7832)
[0. 0. 1. ... 0. 0. 0.]
(2129, 7831) (2129,)
(533, 7832)
[0. 0. 1. ... 0. 0. 0.]
(533, 7831) (533,)
current task: 0 GBT
[LightGBM] [Warning] Unknown parameter: scale_pos_Weight



 80%|████████  | 4/5 [00:30<00:08,  8.05s/it]

auc_score is :  0.7456405097250167
(2129, 7832)
[0. 0. 1. ... 0. 0. 0.]
(2129, 7831) (2129,)
(533, 7832)
[0. 0. 1. ... 0. 0. 0.]
(533, 7831) (533,)
current task: 0 GBT
[LightGBM] [Warning] Unknown parameter: scale_pos_Weight



100%|██████████| 5/5 [00:38<00:00,  7.95s/it]

auc_score is :  0.7456405097250167
